Given six sets of data (three per DSC perfusion scan (denoted by P00#))
These three are:

LONG_EPI-LL_POST --> LL_EPI_POST

LONG_EPI-LL_PRE --> LL_EPI_PRE

FE_EPI --> ep2d_perf

rename by number and sort appropriately. 


CHECK BEFORE RUNNING: Naming convention of the scans directly fromm scanner may change. T1 pre, DSC perfusion, and T1 post are 'prename', 'epname', and 'postname' in func sortDSCfolders. Make sure naming patterns match those defined in that function. 

Mira Liu - 2018

In [1]:
import numpy as np
import glob,os
import shutil
import pydicom as pd
from itertools import permutations
import math


In [2]:
#modified shutil.copytree
def copytree(src, dst, symlinks=False, ignore=None):
    if not os.path.exists(dst):
        os.makedirs(dst)
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            copytree(s, d, symlinks, ignore)
        else:
            if not os.path.exists(d) or os.stat(s).st_mtime - os.stat(d).st_mtime > 1:
                shutil.copy2(s, d)

#input pnum AND the sorted list of faw data folders
def sortDSCfolders(Pnum, RawDataPerms):
    #go through every possible permutation of folders until it's the correct set of 'pre, 'post, and 'ep'
    perm = permutations(range(len(RawDataPerms)))
    for ivals in list(perm): #then try different permutations of folder names
        prename = str(RawDataPerms[ivals[0]]) #is the first one 'pre t1' folder? (etc)
        epname = str(RawDataPerms[ivals[1]])
        postname = str(RawDataPerms[ivals[2]])
        #if we have the correct order of t1 pre and post and dsc
        if 'LONG' in prename and 'PRE' in prename and 'EPI' in prename:
            if 'FE_EPI' in epname:
                if 'LONG' in postname and 'POST' in postname and 'EPI' in postname:
                    #sort them into rsepective folders
                    Pnum = Pnum + 1
                    if not os.path.exists(dirSortedData+'/P00'+str(Pnum)):
                        os.makedirs(dirSortedData+'/P00'+str(Pnum)) #make correct file
                    copytree(dirRawData+'/'+prename,dirSortedData+'/P00'+str(Pnum)+'/IR_LL_EPI_PRE')
                    if len(os.listdir(str(dirRawData)+ '/' + str(epname))) ==1:
                        copytree(dirRawData+'/'+epname+'/30',dirSortedData+'/P00'+str(Pnum)+'/ep2d_perf')
                    else:
                        copytree(dirRawData+'/'+epname,dirSortedData+'/P00'+str(Pnum)+'/ep2d_perf')
                    copytree(dirRawData+'/'+postname,dirSortedData+'/P00'+str(Pnum)+'/IR_LL_EPI_POST')
                    
                    #now remove the sorted names from the possible data folders
                    RawDataPerms.remove(prename)
                    RawDataPerms.remove(epname)
                    RawDataPerms.remove(postname)
                    return Pnum, RawDataPerms #outputs the updated pnum and updated available raw data folders
                
                #input pnum AND the sorted list of faw data folders


# first sort the raw data into three correct folders for post-processing

In [ ]:
#give patient number
folderpath = '/Users/neuroimaging/Desktop/DATA/ICAD/Pt'
ptnum = '10'
print('running pt '+ ptnum)

#give original directory and end directory ()ends in DICOM_sorted usually
dirRawData = folderpath+ptnum+'/pt'+ptnum+'_DSC'
dirSortedData = folderpath+ptnum+'/pt'+ptnum+'_DSC_Processed'
RawData = sorted(os.listdir(str(dirRawData)))
#print(RawData)
if not os.path.exists(dirSortedData):
    os.makedirs(dirSortedData)
print(RawData)

Pnum = 0 #starting with 0
PnumLim = math.floor(len(RawData)/3) #how many folders (i.e. dsc scans) there should be 

#RawData.remove('.DS_Store')
RawDataPerms = RawData
if Pnum < PnumLim: # if we still need to process more dsc scans for this subject
    Pnum,RawDataPerms= sortDSCfolders(Pnum,RawDataPerms)
print('Sorted All files\nNow Number Files using code below')

# now sort the data within these folders by time point and slice

In [ ]:
dirData = folderpath+ptnum+'/pt'+ptnum+'_DSC_Processed/P00'
print('sorting pt ' + ptnum)
Pnum1 = 1

## first starting with t1 pre 
filepathPre = str(dirData)+str(Pnum1)+'/LL_EPI_PRE/'
p001_pre = os.listdir(filepathPre)

#rename so not overwritten
for i in range(len(p001_pre)):
    os.rename(filepathPre+str(p001_pre[i]),filepathPre+ 'old'+str(p001_pre[i]))
p001_pre = os.listdir(filepathPre) #get new directory
for i in range(len(p001_pre)):
    Dcm = pd.read_file(str(filepathPre)+str(p001_pre[i]))
    num =str(Dcm[0x0020,0x0013].value) #slice number in dicom header
    os.rename(filepathPre+str(p001_pre[i]),filepathPre+ num+'.dcm')

## Now starting with t1 post
filepathPost = str(dirData)+str(Pnum1)+'/LL_EPI_POST/'
p001_post = os.listdir(filepathPost)
#rename so not overwritten
for i in range(len(p001_post)):
    os.rename(filepathPost+str(p001_post[i]),filepathPost+ 'old'+str(p001_post[i]))
p001_post = os.listdir(filepathPost) #get new directory
for i in range(len(p001_post)):
    Dcm = pd.read_file(str(filepathPost)+str(p001_post[i]))
    num =str(Dcm[0x0020,0x0013].value) #slice number in dicom header
    os.rename(filepathPost+str(p001_post[i]),filepathPost+ num+'.dcm')
    

    
#NOW reorder the ep2_perf images
filepathPerf = str(dirData)+str(Pnum1)+'/ep2d_perf/'
p001_perf = os.listdir(filepathPerf)

#check how many images there are (more than 8?)
if len(p001_perf[0])< 8:
    for i in range(len(p001_perf)):
        os.rename(filepathPerf+str(p001_perf[i]),filepathPerf+'old' +str(p001_perf[i]))
p001_perf = os.listdir(filepathPerf)

#first count how many slices and time points per slice there are
count = 0
slices = []
for i in range(len(p001_perf)):
    if '.DS_Store' not in str(p001_perf[i]): #remove .DS store file
        Dcm = pd.read_file(str(filepathPerf)+str(p001_perf[i]))
        Imnum =float(Dcm[0x0020,0x0013].value) #slice number in dicom header
        if Imnum not in slices: #if images from this slice haven't been organized yet
            slices.append(Imnum) #get list of all slices
        count = count + 1 #how many total images
        
totalslices = int(max(slices))
timepoints = count/totalslices #how mmany time points per slice


#now start sorting them by slice
names = [[] for _ in range(totalslices)] #create array of empty arrays for slices with timepoints within each sublist
for i in range(len(p001_perf)):
    if '.DS_Store' not in str(p001_perf[i]): #remove .DS store file
        Dcm = pd.read_file(str(filepathPerf)+str(p001_perf[i]))
        Imnum =int(Dcm[0x0020,0x0013].value) #slice number in dicom header
        names[Imnum-1].append(p001_perf[i]) #put each image of the ith slice within the respective i'th array 
    else:
        os.remove(str(filepathPerf)+str(p001_perf[i])) #remove the .DS store file

#now organizing within each slice
for k in range(len(names)): 
    SliceImages = names[k] #get the list of images within the k'th slice
    intz = []
    base = timepoints*k #how many time points per slice
    for i in range(len(SliceImages)): #for the images within the k'th slice
        Dcm = pd.read_file(str(filepathPerf)+str(SliceImages[i]))
        num =float(Dcm[0x0008,0x0032].value) #get the acquisition time 
        intz.append(int(num*100) )
    sortSliceImages = (sorted(intz)) #sort the acquisition time in order
    for i in range(len(SliceImages)): #now reorganize them in order of acquisition time per slice.
        Dcm = pd.read_file(str(filepathPerf)+str(SliceImages[i]))
        num =float(Dcm[0x0008,0x0033].value)
        os.rename(filepathPerf+str(SliceImages[i]),filepathPerf+ str(sortSliceImages.index(int(num*100))+int(base)+1)+'.dcm')
        

print('Done! All ready for post-processing now :)')

